In [ ]:
def build_resnet18_for_cifar10(num_classes: int = 10):
    model = models.resnet18(weights=None)  # ImageNet pretrained 없이 ResNet18 생성 (from scratch)

    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)  # CIFAR용 stem
    model.maxpool = nn.Identity()  # maxpool 제거
    model.fc = nn.Linear(model.fc.in_features, num_classes)  # 최종 분류기 출력=10
    return model

In [ ]:
""" AMP와 AMP용 scalerdls GradScaler를 사용 """

scaler = torch.amp.GradScaler(enabled=torch.cuda.is_available())  
# AMP용 scaler, 학습 속도를 높이고 메모리 사용량을 줄임

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()  # train mode(dropout/bn 동작 변경)
    total_loss, total_acc = 0.0, 0.0  # 누적 loss/acc
    n = 0  # 샘플 수 누적
    
    for images, labels in loader:
        images = images.to(device, non_blocking=True)  # 입력을 GPU로 이동 (Asynchronous transfer)
        labels = labels.to(device, non_blocking=True)  # 라벨을 GPU로 이동 (Asynchronous transfer)
        
        optimizer.zero_grad(set_to_none=True)  # gradient 초기화

        #AMP(Automatic Mixed Precision), 계산이 복잡한 곳은 FP32를 쓰고, 단순한 곳은 FP16을 섞어서 사용
        with torch.amp.autocast(device_type=device.type, enabled=torch.cuda.is_available()):  # AMP autocast
            logits = model(images)  # forward
            loss = criterion(logits, labels)  # loss 계산
        
        scaler.scale(loss).backward()  # scaled backward, 단순 loss.backward()와 다른점 유의
        scaler.step(optimizer)  # optimizer step, scaler 안쓸때는 optimizer.step()
        scaler.update()  # scaler 업데이트, scaler 안쓸때는 사용 안하던 코드인듯.

        bs = images.size(0)  # batch size
        total_loss += loss.item() * bs  # batch loss 누적
        total_acc  += accuracy_top1(logits.detach(), labels) * bs  # logits값만 복사, batch acc 누적
        n += bs  # 샘플 수 누적
    
    return total_loss / n, total_acc / n, dt  # 평균 loss/acc/시간


In [ ]:
@torch.no_grad()   # Autograd off, 함수 내부에서 기울기 계산 멈춤
def evaluate(model, loader, criterion):
    model.eval()  # eval mode, i.e., Dropout 비활성화, BN Running Stats 고정
    total_loss, total_acc = 0.0, 0.0  # 누적
    n = 0  # 샘플 수
    for images, labels in loader:
        images = images.to(device, non_blocking=True)  # GPU 이동
        labels = labels.to(device, non_blocking=True)  # GPU 이동
        logits = model(images)  # forward
        loss = criterion(logits, labels)  # loss, (lo, la) 순서 기억하자.
        bs = images.size(0)  # batch size
        total_loss += loss.item() * bs  # 누적
        total_acc  += accuracy_top1(logits, labels) * bs  # 누적
        n += bs  # 누적
    return total_loss / n, total_acc / n  # 평균 loss/acc

In [ ]:
""" 체크포인트로 실험을 이어가는 코드 봐두기 """
criterion = nn.CrossEntropyLoss(label_smoothing=cfg.label_smoothing)  # 분류 loss
optimizer = torch.optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.momentum, weight_decay=cfg.weight_decay)  # Mini-Batch SGD optimizer with Momentum

ckpt_path = 'resnet18_cifar10.pth'  # best checkpoint 경로
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}  # 기록용 dict

start_epoch = 0  # resume 시작 epoch
best_acc = -1.0  # best val acc

if os.path.exists(ckpt_path):  # 체크포인트가 있으면 resume
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)  # checkpoint 로드
    model.load_state_dict(ckpt['model'])  # 모델 가중치 로드
    start_epoch = int(ckpt.get('epoch', 0))  # 저장된 epoch
    best_acc = float(ckpt.get('val_acc', -1.0))  # 저장된 best acc
    epochs_to_run = cfg.extra_epochs_if_resume  # 추가 실험 epoch = 10
else:  # 체크포인트가 없으면 처음부터
    epochs_to_run = cfg.base_epochs_if_new  # 신규 실험 epoch = 20

# 학습률을 T_max에 맞춰 코사인 곡선 형태로 변화
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs_to_run)  
# 이번 실험 구간에 대해 cosine schedule

for e in range(1, epochs_to_run+1):
    epoch = start_epoch + e  # 실제 epoch 번호(누적)
    lr_now = optimizer.param_groups[0]['lr']  # 현재 lr, [ { 'params': [...], 'lr': 0.1, 'momentum': 0.9, 'weight_decay': 0.0005, ... } ]
    tr_loss, tr_acc, dt = train_one_epoch(model, train_loader, optimizer, criterion)  # 1 epoch train
    va_loss, va_acc = evaluate(model, test_loader, criterion)  # validation
    scheduler.step()  # lr 스케줄 업데이트

    history['train_loss'].append(tr_loss)  # 기록
    history['train_acc'].append(tr_acc)  # 기록
    history['val_loss'].append(va_loss)  # 기록
    history['val_acc'].append(va_acc)  # 기록
    history['lr'].append(lr_now)  # 기록
    
    if va_acc > best_acc:  # best 갱신 시 저장
        best_acc = va_acc  # best 업데이트
        torch.save({'model': model.state_dict(), 'epoch': epoch, 'val_acc': va_acc, 'config': cfg.__dict__}, ckpt_path)  # 체크포인트 저장


In [ ]:
""" 추론(Inference)
    - 학습된 모델이 실제로 어떤 클래스로 예측하는지 확인
    - softmax 확률과 confidence(최대 확률) 부분 잘 보기
"""

ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)  # best checkpoint 로드
model.load_state_dict(ckpt['model'])  # 최신 모델 가중치 적용
model.eval()  # eval 모드, i.e., Dropout 비활성화, BN Running Stats 고정

@torch.no_grad()  # 자동 미분(Autograd) 기능 비활성화
def predict_batch(model, images):
    logits = model(images)  # raw logits
    probs = F.softmax(logits, dim=1)  # 확률로 변환, dim=0 은 batch 차원
    conf, pred = probs.max(dim=1)  # 최대 확률(conf)과 클래스(pred)
    return pred, conf, probs

In [ ]:
""" confusion matrix 
    그냥 preds, labels, images 를 cpu로 보내는것 정도만 봐도 될듯?
"""
@torch.no_grad()
def collect_predictions(model, loader):
    model.eval()  # eval 모드
    all_preds, all_labels, all_images = [], [], []  # 누적 리스트
    for images, labels in loader:
        images = images.to(device)  # GPU 이동
        labels = labels.to(device)  # GPU 이동
        preds, conf, _ = predict_batch(model, images)  # 예측
        all_preds.append(preds.cpu())  # CPU로 모아두기
        all_labels.append(labels.cpu())  # CPU로 모아두기
        all_images.append(images.cpu())  # 이미지도 저장(오분류 시각화용)
    return torch.cat(all_images), torch.cat(all_preds), torch.cat(all_labels)  # 전체 텐서로 결합

In [ ]:
"""
    per_class_acc 매트릭스 연산 정도만 체크해도?
"""
def confusion_and_perclass(preds, labels, num_classes):
    cm = torch.zeros((num_classes, num_classes), dtype=torch.int64)  # confusion matrix 초기화
    for t, p in zip(labels, preds):
        cm[int(t), int(p)] += 1  # GT=t, Pred=p 카운트 증가
    per_class_acc = cm.diagonal().float() / torch.clamp(cm.sum(dim=1).float(), min=1.0)  # 클래스별 정확도
    return cm, per_class_acc